In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [3]:
data = pd.read_csv('data_v3/train_data_v3.csv', parse_dates=True)

In [4]:
data

,time,Measure,kWh,label
0,2022-01-03 00:00:00,Mech_Svc_AHU,8.0000,unclassified
1,2022-01-03 00:00:00,Sub - Main M2,1000.0000,unclassified
2,2022-01-03 00:00:00,FB1 - Main 1L,26.5000,unclassified
3,2022-01-03 00:00:00,AB3 - Main 3L,36.7500,Non-production
4,2022-01-03 00:00:00,Sub - Feeder F04,0.0000,unclassified
...,...,...,...,...
652419,2022-11-13 23:45:00,Sub - Feeder F03,22.1250,unclassified
652420,2022-11-13 23:45:00,Sub - Feeder F04,0.0000,unclassified
652421,2022-11-13 23:45:00,Paint_AHU,8.2500,unclassified
652422,2022-11-13 23:45:00,BS4 - Main 4L,47.5000,unclassified


In [7]:
data1 = pd.get_dummies(data, columns=['label'])
data1

,time,Measure,kWh,label_Non-production,label_Power-down,label_Power-up,label_Production,label_unclassified
0,2022-01-03 00:00:00,Mech_Svc_AHU,8.0000,0,0,0,0,1
1,2022-01-03 00:00:00,Sub - Main M2,1000.0000,0,0,0,0,1
2,2022-01-03 00:00:00,FB1 - Main 1L,26.5000,0,0,0,0,1
3,2022-01-03 00:00:00,AB3 - Main 3L,36.7500,1,0,0,0,0
4,2022-01-03 00:00:00,Sub - Feeder F04,0.0000,0,0,0,0,1
...,...,...,...,...,...,...,...,...
652419,2022-11-13 23:45:00,Sub - Feeder F03,22.1250,0,0,0,0,1
652420,2022-11-13 23:45:00,Sub - Feeder F04,0.0000,0,0,0,0,1
652421,2022-11-13 23:45:00,Paint_AHU,8.2500,0,0,0,0,1
652422,2022-11-13 23:45:00,BS4 - Main 4L,47.5000,0,0,0,0,1


In [10]:
data.label.unique()

array(['unclassified', 'Non-production', 'Power-up', 'Production',
       'Power-down'], dtype=object)

In [11]:
d = {'unclassified':0, 'Non-production':1, 'Power-up':2, 'Production':3, 'Power-down':4}

In [12]:
data.replace({'label': d}, inplace=True)

In [14]:
X = data1.drop(['time', 'Measure'], axis=1).values.astype('float32')
y = data['label'].values.astype('float32')

In [15]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.45, random_state=101)

In [39]:
print(X_train.shape)
print(y_train.shape)

(358833, 6)
(358833,)


In [40]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout

In [42]:
model = Sequential()
model.add(Dense(units=16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=4, activation='relu'))
model.add(Dense(units=1, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
22428/22428 [==============================] - 22s 953us/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch 2/10
22428/22428 [==============================] - 22s 975us/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch 3/10
22428/22428 [==============================] - 21s 951us/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch 4/10
22428/22428 [==============================] - 23s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch 5/10
22428/22428 [==============================] - 21s 937us/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch 6/10
22428/22428 [==============================] - 20s 902us/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch 7/10
22428/22428 [==============================] - 21s 931us/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch 8/10
22428/22428 [==============================] - 22s 992us/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch 9/10
22428/22428 [==============================] - 22s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0534
Epoch

In [43]:
loss, accuracy = model.evaluate(x=X_test, y=y_test, batch_size=5)

print("Test loss:", loss)
print("Test accuracy:", accuracy)

58719/58719 [==============================] - 48s 809us/step - loss: 0.0000e+00 - accuracy: 0.0530
Test loss: 0.0
Test accuracy: 0.05296824499964714


In [44]:
final = pd.read_csv('data_v3/holdout_v3.csv', parse_dates=True)

In [45]:
final

,time,Measure,kWh,label
0,2020-07-27 00:00:00,Sub - Feeder F08,481.500,Non-production
1,2020-07-27 00:15:00,Sub - Feeder F08,481.375,Non-production
2,2020-07-27 00:30:00,Sub - Feeder F08,481.250,Non-production
3,2020-07-27 00:45:00,Sub - Feeder F08,481.125,Non-production
4,2020-07-27 01:00:00,Sub - Feeder F08,481.000,Non-production
...,...,...,...,...
2683,2020-09-20 22:45:00,Sub - Feeder F08,372.125,Non-production
2684,2020-09-20 23:00:00,Sub - Feeder F08,371.500,Non-production
2685,2020-09-20 23:15:00,Sub - Feeder F08,378.125,Non-production
2686,2020-09-20 23:30:00,Sub - Feeder F08,384.750,Non-production


In [47]:
final.label.unique()

array(['Non-production', 'Power-up', 'Production', 'Power-down'],
      dtype=object)